### Лабораторная работа 2: Задание 6.2, пункт 1), стр. 175

По 1000 реализаций $\mathit{S}$ вычисленных по формуле (1), оценить числовые 
характеристики $\mathit{S}: S_{min}, S_{max}, E\{S\}, D\{S\}, P\{S_1 ≤ S ≤ S_2 \}$, для следующих 
значений параметров, входящих в правую часть формулы (1):

$\mathit{i_3}$ |8.6 %|8.7 %|8.8 %|9.2 %
-----|-----|-----|-----|-----
$\mathit{P}$|0.1|0.3|0.5|0.1

$\mathit{i_4}$ |8.7 %|8.9 %|9.4 %
-----|-----|-----|-----
$\mathit{P}$|0.1|0.6|0.3

Формула (1):
$S = P(1 + n_1i_1 + n_2i_2 + ... + n_ki_k)$                                                    



In [1]:
import numpy as np
import pandas as pd
from math import floor
from scipy.stats import chi2
import time

import typing as tp

from tabulate import tabulate

##### Мультипликативный конгруэнтный метод моделирования БСВ 

In [2]:
def generate_brv_congruential_sample(M: int=2**31, alpha_star0: int=65539, beta :int=65539) -> float:
    alpha_t = alpha_star0
    while True:
        alpha_t = (alpha_t * beta) % M
        yield alpha_t / M
        
def generate_brv_congruential(n: int=100, M: int=2**31, alpha_star0: int=65539, beta :int=65539) -> np.ndarray: 
    alpha = np.array([])
    generator = generate_brv_congruential_sample(M=M, alpha_star0=alpha_star0, beta=beta)
    for i in range(n): 
        alpha = np.append(alpha, next(generator))
    return alpha

##### Метод Макларена-Марсальи моделирования БСВ для моделирования равномерного распределения на отрезке 

In [3]:
def generate_brv_mm_sample(k: int=128, a:float=0, b:float=1) -> float:
    v = np.array([])
    brv_cong_b = generate_brv_congruential_sample()
    t = time.perf_counter()
    alpha_star0 = beta =  int(10**9*float((t-int(t))))
    brv_cong_c = generate_brv_congruential_sample(alpha_star0=alpha_star0, beta=beta)
    for i in range(k):
            v = np.append(v, next(brv_cong_b))
    while True:
        index = floor(next(brv_cong_c) * k)
        alpha_t = v[index]
        v[index] = next(brv_cong_b)
        yield alpha_t * (b - a) + a
        
def generate_brv_mm(n: int=100, k: int=128,  a:float=0, b:float=1) -> np.ndarray: 
    alpha = np.array([])
    generator = generate_brv_mm_sample(k=k, a=a, b=b)
    for i in range(n):
        alpha = np.append(alpha, next(generator))
    return alpha

##### Алгоритм моделирования ДСВ (для выбора $i_3$ и $i_4$)

In [4]:
def random_choice(array: list=None, size: int=1, probas: list=None) -> list:
    probas_cum = []
    last_element = 0
    for i in range(len(probas)):
        probas_cum.append(last_element + probas[i])
        last_element = probas_cum[i]
    
    generator = generate_brv_mm_sample()
    
    result = []
    for i in range(size):
            rand_uniform = next(generator)
            for j in range(len(probas_cum)):
                if(rand_uniform < probas_cum[j]):
                    result.append(array[j])
                    break
    return result if size > 1 else result[0]

### Основной цикл программы

In [5]:
k = 4
n = 0.25
P = 1230
S_interval = [1335, 1340]
R = {"i_1": (8.2, 9), "i_2" : (8.4, 9.1)}
probas = {"i_3": {8.6: 0.1, 8.7: 0.3, 8.8: 0.5, 9.2: 0.1}, "i_4": {8.7: 0.1, 8.9: 0.6, 9.3: 0.3}}

N = 1000
S = np.ones(N)

i_param_full = []

for j in range(N):
    i_arr = []
    for i in range(k):
        if i == 0 or i == 1:
            generator = generate_brv_mm_sample(a=R[f"i_{i+1}"][0], b=R[f"i_{i+1}"][1])
            i_arr.append(next(generator))
        else:
            i_arr.append(random_choice(list(probas[f"i_{i+1}"].keys()), \
                                          size=1, probas=list(probas[f"i_{i+1}"].values())))
        S[j] += n * i_arr[i] / 100
    i_param_full.append(i_arr)
    S[j] *= P

### Получены 1000 реализаций  вектора $i = (i_1, i_2, i_3, i_4)$ 
##### Показаны первые 10

In [6]:
print(tabulate(i_param_full[:10], headers=['i_1', 'i_2', 'i_3', 'i_4'], floatfmt=".4f"))

   i_1     i_2     i_3     i_4
------  ------  ------  ------
8.7425  8.7333  8.6000  8.7000
8.4554  8.9785  8.8000  9.3000
8.3795  8.8747  8.8000  8.9000
8.7277  8.9193  8.8000  9.3000
8.9925  8.6256  8.8000  8.9000
8.8290  8.6141  8.8000  8.9000
8.2202  8.7418  8.8000  8.9000
8.8516  9.0365  8.6000  9.3000
8.9131  8.8359  8.8000  9.3000
8.6272  8.7709  8.7000  8.9000


### Получены 1000 реализаций  величины $S$
##### Показаны первые 10

In [7]:
print(S[:10])

[1336.93552085 1339.2665159  1337.48414561 1339.92182406 1338.60322711
 1338.06503053 1336.58551519 1340.04841013 1340.23552741 1337.61904064]


### $S_{min}$

In [8]:
S.min()

1334.8673816353535

### $S_{max}$

In [9]:
S.max()

1342.3111569800492

### $E\{S\}$

In [10]:
S.mean()

1338.3612634573874

### $D\{S\}$

In [11]:
S.var()

1.8736606525666706

### $P\{S_1 ≤ S ≤ S_2 \}$

In [12]:
len([s for s in S if s <= S_interval[1] and s >= S_interval[0]]) / len(S)

0.878